In [74]:
import numpy as np
import xmlTranslator as xmlT
from tqdm import tqdm
%reload_ext autoreload
%autoreload 2
import importlib
from utils import get_project_root
from ipywidgets import FloatProgress

TODO: 
- Repeat the sequences 3 times
- Clean all vocabulary into a concise repertoire 
- Add the maj nature
- Extend all chords into 18 enharmonic
- Split into train and test
- create the new MIDI array 
- Pass the MIDI into Embeddings

In [21]:
directory = '/data/iRealXML'
root = get_project_root()
myPath = str(root) + str(directory)

theChordDataset, theOffsetDataset, all_meta = xmlT.parse_info_from_XML(myPath)
print(theChordDataset.shape, theOffsetDataset.shape, len(all_meta))

100%|██████████| 4007/4007 [00:17<00:00, 229.18it/s]

(4006,) (4006,)
(4006,) (4006,) 4006


In [22]:
importlib.reload(xmlT)

id = 0
expand_form_dataset = []
status = True
for i in tqdm(range(theChordDataset.shape[0])):
    song_structure, status = xmlT.expand_song_structure(theChordDataset[i], i)
    if status == False:
        print('Song:', all_meta[i])
        id = i
        break
    song_structure = np.array(song_structure)
    expand_form_dataset.append(song_structure)


100%|██████████| 4006/4006 [00:01<00:00, 3965.98it/s]


In [9]:
import importlib
importlib.reload(xmlT)
id = 287
print(all_meta[id])
song_structure, status = xmlT.expand_song_structure(theChordDataset[id], 41)

seq = theChordDataset[id].tolist()
print(seq)
print(song_structure)

{'composer': 'Chico Buarque', 'style': 'Samba', 'song_name': 'Apesar de Você  - In AB ABBB In 1', 'tonality': 'A major', 'midi_key': 21, 'time_signature': '4/4', 'decade': 'Null'}
['<style>', 'Samba', '|:', '.', 'A', '69', 'add 9 alter #5', '.', 'E', '7', 'add 9 alter #5', '|', '.', 'A', '69', 'add 9 alter #5', '.', 'E', '7', 'add 9 alter #5', '|', '.', 'A', '69', 'add 9 alter #5', '.', 'E', '7', 'add 9 alter #5', '|', 'Repeat_1', '.', 'A', '69', 'add 9 alter #5', '.', 'E', '7', 'add 9 alter #5', ':|', '|', 'Repeat_2', '.', 'A', '69', 'add 9', '.', 'D', '7', '.', 'C#', '7', '|', '.', 'F#', 'm7', '.', 'F', '|', '.', 'C#', 'm7', 'alter b5', '.', 'F#', '7', '|', '.', 'B', '9', '.', 'E', '7', '|', '.', 'A', '69', 'add 9 alter #5', '.', 'C#', '7', 'add 9 alter #5', '|', '.', 'F#', 'm7', '.', 'F', '|', '.', 'C#', 'm7', 'alter b5', '|', '.', 'C#', 'm7', 'alter b5', '|', '.', 'F#', '7', '|', '.', 'B', '9', '.', 'E', '7', '|', '.', 'A', '6', '/', 'C', '.', 'D', 'm6', '|', '.', 'A', '6', '/', 'C

In [23]:
#save the expanded dataset

max = {'v': 0, 'id': 0}
min = {'v': 1000, 'id': 0}

for i in range(len(expand_form_dataset)):
    if len(expand_form_dataset[i]) > max['v']:
        max['v'] = len(expand_form_dataset[i])
        max['id'] = i
    if len(expand_form_dataset[i]) < min['v']:
        min['v'] = len(expand_form_dataset[i])
        min['id'] = i

print('Max:', max, 'Min:', min)


Max: {'v': 1321, 'id': 141} Min: {'v': 30, 'id': 492}


In [24]:
#Repeat the sequences 3 times
concatenated_array = []

concatenated_array = np.hstack(expand_form_dataset) 

vocab = set(concatenated_array)

print(len(vocab))
print(vocab)

372
{'add b6', 'Up Swing', 'Medium Slow', 'alter b5 add #11', 'Folk-Rock', 'dim(maj7)', 'Soul', 'm69', 'add b13', '7b5#5*', 'add 9 add b13', 'sus2', '*ø11*', 'add b9 add 4 subtract 3 add #9 alter #5', 'Funk', 'Waltz', 'alter b9 alter b9', 'add #11 add 7', 'add b9 add b13', 'add #11 add #9 alter #5', 'add 4 subtract 3 add #9 alter #5', 'Bolero-Cha', 'add #9 add b9', "Even 8's", 'Samba Enredo', 'add #11', 'add #11 add #9', 'Gary Aprile', 'alter #5 alter b5', 'G', 'Db', 'm(add9)', 'Even 8ths', 'Folk Rock', '6b5*', 'Pop Rock', 'B', 'Pop Folk', 'Form_Segno', 'Medium Up Swing', 'alter b5 add 4 subtract 3', 'Afro', 'add 4 subtract 3 add b9', 'A#', 'G*', 'add b13 add #9', 'Slow Rock', 'add #11 add b9 add #11', 'add #9', 'alter #5 add #11', 'Samba-Funk', 'Latin-Swing', 'Cb', 'add b9 alter #5', 'm6', 'add b9 add b9 add 9', 'add 4 subtract 3 alter #11', 'Reggae', 'maj9', 'Salsa', 'add #11 add b9', 'Medium Rock', 'add 2 add #9', 'alter #5 add b13', 'add b9 add #9', 'N.C.', '*m7*', 'add b9 add b9 a

In [60]:
#find orm_Head
id = 0
find = "G7"

for i in range(len(expand_form_dataset)):
    if find in expand_form_dataset[i]:
        print(i)
        print(expand_form_dataset[i])
        id = i
        break
    
print(all_meta[id])

# missing m_maj7, o, aug, aug7
# natures = ['maj', 'sus7', 'm', 'm11', 'm6', 'm7', 'm9', 'dom_7', 'maj7', 'o7', 'sus4', 'sus2', 'ø7', power]


#TODO: fix 'F' 'orm_Bass' and 'F' 'orm_Head'

1303
['<style>' 'Waltz' '|' 'Form_A' '.' 'G' 'maj7' '|' '.' 'B' 'm7' '.' 'G'
 '/' 'B' '|' '.' 'C#' 'm7' '.' 'F#' '7' 'add b9' '|' '.' 'F#' '7' 'add b9'
 '/' 'B' '.' 'B' 'm' '|' '.' 'A' 'm7' '.' 'D' '7sus4' 'add 7' '.' 'D' '7'
 '|' '.' 'F' '/' 'G' '.' 'G' '.' 'F' '/' 'G' '|' '.' 'C' 'add 9' '|' '.'
 'G' 'sus4' '.' 'G' '.' 'G' '7' '|' '.' 'C' 'sus4' '.' 'C' '|' '.' 'E'
 'm7' '.' 'C' '/' 'E' '|' '.' 'Ab' '.' 'Bb' '|' '.' 'Db' '/' 'E' '.' 'Eb'
 '.' 'Db' '/' 'E' '|' '.' 'Ab' 'add 9' '|' '.' 'G7' '#5#9' '|' '.' 'Gb'
 '69' 'add 9' '|' '.' 'G' 'ø7' '/' 'D' '.' 'C' '7' 'add b9' '|' 'Form_B'
 '.' 'F' 'm9' '|' '.' 'C' 'm9' '|' '.' 'G' 'm11' '|' 'Form_Coda' '.' 'Ab'
 'maj7' '.' 'Ab' '7' '|' '.' 'C' 'sus4' '.' 'C' '|' '.' 'Ab' 'maj7' '.'
 'Bb' '/' 'A' '|' '.' 'C' 'sus4' '|' '.' 'C' '.' 'D' 'Form_A' '.' 'G'
 'maj7' '|' '.' 'B' 'm7' '.' 'G' '/' 'B' '|' '.' 'C#' 'm7' '.' 'F#' '7'
 'add b9' '|' '.' 'F#' '7' 'add b9' '/' 'B' '.' 'B' 'm' '|' '.' 'A' 'm7'
 '.' 'D' '7sus4' 'add 7' '.' 'D' '7' '|' '.' 'F' '

In [107]:
importlib.reload(xmlT)

corrected_sequence = xmlT.repleaceTheseChords(expand_form_dataset, False)

my_seq = np.array(corrected_sequence[0])

#compare the original and the new sequence
print(my_seq)

100%|██████████| 2/2 [00:00<00:00, 326.79it/s]

['<style>' 'Waltz' '|' 'Form_A' '.' 'G' 'maj7' '|' '.' 'B' 'm7' '.' 'G'
 '/' 'B' '|' '.' 'C#' 'm7' '.' 'F#' 'dom7' 'add b9' '|' '.' 'F#' 'dom7'
 'add b9' '/' 'B' '.' 'B' 'm' '|' '.' 'A' 'm7' '.' 'D' 'sus7' 'add 7' '.'
 'D' 'dom7' '|' '.' 'F' '/' 'G' '.' 'G' '.' 'F' '/' 'G' '|' '.' 'C'
 'add 9' '|' '.' 'G' 'sus4' '.' 'G' '.' 'G' 'dom7' '|' '.' 'C' 'sus4' '.'
 'C' '|' '.' 'E' 'm7' '.' 'C' '/' 'E' '|' '.' 'Ab' '.' 'Bb' '|' '.' 'Db'
 '/' 'E' '.' 'Eb' '.' 'Db' '/' 'E' '|' '.' 'Ab' 'add 9' '|' '.' 'G' 'dom7'
 'add #5 add b9' '|' '.' 'Gb' 'maj add 6 add 9' 'add 9' '|' '.' 'G' 'ø7'
 '/' 'D' '.' 'C' 'dom7' 'add b9' '|' 'Form_B' '.' 'F' 'm7 add 9' '|' '.'
 'C' 'm7 add 9' '|' '.' 'G' 'm add 11' '|' 'Form_Coda' '.' 'Ab' 'maj7' '.'
 'Ab' 'dom7' '|' '.' 'C' 'sus4' '.' 'C' '|' '.' 'Ab' 'maj7' '.' 'Bb' '/'
 'A' '|' '.' 'C' 'sus4' '|' '.' 'C' '.' 'D' 'Form_A' '.' 'G' 'maj7' '|'
 '.' 'B' 'm7' '.' 'G' '/' 'B' '|' '.' 'C#' 'm7' '.' 'F#' 'dom7' 'add b9'
 '|' '.' 'F#' 'dom7' 'add b9' '/' 'B' '.' 'B' 'm' '|'